In [1]:
%pip install numpy pandas matplotlib tensorflow scikit-learn setuptools keras-tuner

In [2]:
#run once
import os
os.chdir('../')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

In [4]:
#Data Setup
data = pd.read_csv('Data\Building_Data_Preprocessed.csv')

#Drop first header row
data = data.drop(0)

X = data.drop('GHGEmissionsIntensity', axis = 1)
y = data['GHGEmissionsIntensity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#train_tensor = tf.convert_to_tensor(X_train, dtype = tf.float32)
#valid_tensor = tf.convert_to_tensor(X_test, dtype = tf.float32)

#train_labels = tf.convert_to_tensor(y_train, dtype = tf.float32)
#valid_labels = tf.convert_to_tensor(y_test, dtype = tf.float32)

<>:2: SyntaxWarning: invalid escape sequence '\B'
<>:2: SyntaxWarning: invalid escape sequence '\B'
C:\Users\jfsal\AppData\Local\Temp\ipykernel_20312\483572453.py:2: SyntaxWarning: invalid escape sequence '\B'
  data = pd.read_csv('Data\Building_Data_Preprocessed.csv')


In [6]:
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from kerastuner import HyperModel
from kerastuner.tuners import Hyperband
from tensorflow.keras.metrics import RootMeanSquaredError


class GHGHyperModel(HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Input(shape=(X_train.shape[1],)))
        model.add(layers.BatchNormalization())

        for i in range(hp.Int('num_layers', 1, 5)):
            units = hp.Int('units_' + str(i), 
                           min_value = 64,
                           max_value = 128,
                           step = 64)
            # Define activation as a hyperparameter
            activation = hp.Choice('activation_' + str(i), values=['relu', 'elu', 'leaky_relu'])
            
            # Add regularization
            kernel_regularizer = regularizers.l2(hp.Float('l2_regularization_' + str(i), 
                                                          min_value=1e-7, 
                                                          max_value=1e-2, 
                                                          sampling='LOG'))

            model.add(layers.Dense(units=units,
                                   activation=activation,
                                   kernel_regularizer=kernel_regularizer))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(hp.Float('dropout_' + str(i),
                                              min_value=0.2,
                                              max_value=0.9,
                                              step=0.1)))
        model.add(layers.Dense(1, activation='linear'))

        model.compile(
            optimizer=keras.optimizers.Adam(
                learning_rate=hp.Float('learning_rate', 
                                       min_value=1e-7,
                                       max_value=1e-3,
                                       sampling='LOG')
            ),
            loss='mse',
            metrics=['mean_absolute_error', RootMeanSquaredError()]
        )
        return model
    
tuner = Hyperband(
    GHGHyperModel(),
    objective = 'val_mean_absolute_error',
    max_epochs = 100,
    factor = 3,
    directory = 'House Price Tuning',
    project_name = 'House Price Prediction',
    overwrite = True
)

#********************#
MY_PATIENCE = 10
MY_EPOCHS = 260
MY_MIN_DELTA = 0.001
#********************#

In [7]:
tuner.search(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs = MY_EPOCHS,
    callbacks = [keras.callbacks.EarlyStopping(patience=MY_PATIENCE, min_delta=MY_MIN_DELTA, restore_best_weights=True)],
    verbose = 1
)

best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparams = tuner.get_best_hyperparameters(num_trials=1)[0]

best_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error', RootMeanSquaredError()])
best_model.evaluate(X_test, y_test)

Trial 2 Complete [00h 00m 01s]

Best val_mean_absolute_error So Far: None
Total elapsed time: 00h 00m 02s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
5                 |2                 |num_layers
64                |64                |units_0
leaky_relu        |elu               |activation_0
9.1264e-07        |3.8488e-06        |l2_regularization_0
0.3               |0.6               |dropout_0
0.00010929        |1.4396e-07        |learning_rate
64                |64                |units_1
leaky_relu        |relu              |activation_1
0.0034218         |1e-07             |l2_regularization_1
0.4               |0.2               |dropout_1
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round



Traceback (most recent call last):
  File "c:\Users\jfsal\Documents\DATA422_Fall2024_Team5\.venv\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\jfsal\Documents\DATA422_Fall2024_Team5\.venv\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jfsal\Documents\DATA422_Fall2024_Team5\.venv\Lib\site-packages\keras_tuner\src\tuners\hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jfsal\Documents\DATA422_Fall2024_Team5\.venv\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwa

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\jfsal\Documents\DATA422_Fall2024_Team5\.venv\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\jfsal\Documents\DATA422_Fall2024_Team5\.venv\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jfsal\Documents\DATA422_Fall2024_Team5\.venv\Lib\site-packages\keras_tuner\src\tuners\hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jfsal\Documents\DATA422_Fall2024_Team5\.venv\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_tria

In [ ]:
#Plot Training and validation loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()